In [12]:
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm_gui
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset,DataLoader
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype=torch.FloatTensor

In [8]:
sentences = ["jack like dog", "jack like cat", "jack like animal",
  "dog cat animal", "banana apple cat dog like", "dog fish milk like",
  "dog cat animal like", "jack like apple", "apple like", "jack like banana",
  "apple banana jack movie book music like", "cat dog hate", "cat dog like"]
sentences_list=" ".join(sentences).split()
vocab=list(set(sentences_list))
word2idx={w:i for i,w in enumerate(vocab)}
vocab_size=len(vocab)

In [4]:
# model parameters
C=2
batch_size=8
m=2

In [10]:
skip_grams=[]
for idx in range(C,len(sentences_list)-C):
    center=word2idx[sentences_list[idx]]
    context_idx=list(range(idx-C,idx))+list(range(idx+1,idx+C+1))
    context=[word2idx[sentences_list[i]] for i in context_idx]

    for w in context:
        skip_grams.append([center,w])

In [11]:
def make_data(skip_grams):
    x,y=[],[]
    for a,b in skip_grams:
        x.append(np.eye(vocab_size)[a])
        y.append(b)
    return x,y

In [14]:
x,y=make_data(skip_grams)
x,y=torch.Tensor(x),torch.Tensor(y)
dataset=TensorDataset(x,y)
loader=DataLoader(dataset,batch_size,True)

In [16]:
class Word2Vec(nn.Module):
    def __init__(self):
        super(Word2Vec,self).__init__()
        self.W=nn.Parameter(torch.randn(vocab_size,m).type(dtype))
        self.V=nn.Parameter(torch.randn(m,vocab_size).type(dtype))
    def forward(self,x):
        hidden=torch.mm(x,self.W)
        output=torch.mm(hidden,self.V)
        return output

In [ ]:
model=Word2Vec().to(device)
loss_fn=nn.CrossEntropyLoss().to(device)